In [56]:
import pandas as pd

### 1. concat 데이터 불러오기

297243 rows × 168 columns

In [57]:
df = pd.read_csv('../외감데이터/dataset_외감.csv',index_col=0)

In [58]:
df = df[df['회계년도'].str.endswith('12')]

### 2. 금융업종 제외하기

산업명 : 금융, 부동산 임대 및 공급업
회사명 : 투자회사, 부동산투자회사, 부동산, 금융투자주식회사, 선박투자회사, 투자주식회사, 투자개발, 금융투자

투자 포함된 회사 검색 후 삭제 (주)건설투자, (주)국제선박투자운용 등등...
리츠 포함된 회사 검색 후 삭제

257351 rows × 168 columns

In [59]:
# 산업 업종 분류가 잘못된 데이터 고려
# 투자, 리츠 포함된 회사는 따로 찾아서 검색 후 삭제

df= df[~df['산업명'].str.contains('금융')]
df= df[~df['산업명'].str.contains('부동산 임대 및 공급업')]
df= df[~df['회사명'].str.contains('투자회사')]
df= df[~df['회사명'].str.contains('부동산투자회사')]
df= df[~df['회사명'].str.contains('부동산')]
df= df[~df['회사명'].str.contains('금융투자주식회사')]
df= df[~df['회사명'].str.contains('선박투자회사')]
df= df[~df['회사명'].str.contains('투자회사')]
df= df[~df['회사명'].str.contains('투자주식회사')]
df= df[~df['회사명'].str.contains('투자개발')]
df= df[~df['회사명'].str.contains('금융투자')]

In [60]:
# 투자 삭제할 회사명 리스트

companies_to_remove = [
    '(주)건설투자', '(주)국제선박투자운용', '주식회사아프로투자',
    '주식회사인천투자펀드2', '주식회사케이티투자운용', '주식회사테라투자파트너스', '(주)케이리츠투자운용',
    '케이디기술투자(주)', '케이엔씨개발투자컨설팅', '(유)비티투자',
    '신동해투자일호피에프브이(주)', '아이비투자검단(주)', '에이스투자(주)', '유한회사한일자산관리앤투자',
    '주식회사건인투자', '주식회사세원투자산업'
]

# 데이터프레임에서 해당 회사명을 가진 행들을 제거

df = df[~df['회사명'].isin(companies_to_remove)]

In [61]:
# 리츠 삭제할 회사명 리스트

companies_to_remove = ['굿모닝대신리츠주식회사', '주식회사평안리츠',  '(주)청리츠앤파트너스',
    '주식회사힘내라리츠', '케이리츠주식회사', '대영리츠건설(주)', '(주)리츠산업',
    '(주)리츠홀딩스',  '신한리츠운용주식회사',
    '월드리츠개발주식회사', '주식회사경성리츠', '주식회사글리츠리츠', '주식회사리츠파크', '주식회사무등리츠',
    '주식회사비앤에스리츠']

# 데이터프레임에서 해당 회사명을 가진 행들을 제거

df = df[~df['회사명'].isin(companies_to_remove)]

### 3. 결측치가 너무 많은 데이터 삭제 (5000개 이상), 컬럼명 통일

257351 rows × 135 columns

In [62]:
# 필요 없는 컬럼제거

df.drop(columns=['상장일','상장폐지일','업종코드','소속코드','감사의견','[제조]종업원1인당 부가가치(백만원)','[제조]종업원1인당 매출액(백만원)',
                '[제조]종업원1인당 경상이익 (2007년 이전 발생)(백만원)','[제조]종업원1인당 순이익(백만원)','[제조]종업원1인당 인건비(백만원)',
                '[제조]노동장비율','[제조]기계장비율','[제조]자본집약도','[제조]원','[제조]재공품회전률','[제조]종업원수','[제조]종업원1인당 부가가치(백만원).1',
                '[제조]총자본투자효율.1','[제조]기계투자효율.1','[제조]부가가치율.1','[제조]부가가치(백만원).1','[제조]경상이익증가율 (2007년 이전 발생)',
                '[제조]매출액경상이익률 (2007년 이전 발생)','[제조]총자본경상이익률 (2007년 이전 발생)','[제조]자기자본경상이익률 (2007년 이전 발생)',
                '[제조]자본금경상이익률 (2007년 이전 발생)','[제조]기업경상이익률 (2007년 이전 발생)','[제조]투자수익률 (2007년 이전 발생)','[제조]종업원1인당 부가가치증가율', 
                '[제조]종업원수증가율', '[제조]종업원1인당 매출액증가율', '[제조]종업원1인당 인건비증가율','[제조]1주당경상이익 (2007년 이전 발생)(원)'],inplace=True)

In [63]:
# 컬럼명 가독성

df.columns=[col.replace('(백만원)','') for col in df.columns]
df.columns=[col.replace('(원)','') for col in df.columns]
df.columns=[col.replace('[제조]','') for col in df.columns]

### 4. 재무비율 생성을 위해 2년 연속 재무제표가 존재하지 않으면 제거

257351 rows × 135 columns

In [64]:
# 회계년도를 시간 데이터로 변경
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%Y/%m')

# 거래소코드와 회계년도 기준으로 정렬
df = df.sort_values(by=['거래소코드', '회계년도'])

# 3년 연속 데이터 체크
def has_consecutive_years(group):
    # year 만 가져오기
    years = group['회계년도'].dt.year
    # 현재 year에서 전 yeear을 뺀값이 1, 총합이 2이상인 것을 반환
    return (years.diff().fillna(1) == 1).cumsum().max() >= 2

# 거래소코드를 기준으로 groupby 후 filter
df = df.groupby('거래소코드').filter(has_consecutive_years)

### 5. Labeling을 위해 자본잠식률 추가

자본잠식률, 이자보상배율 추가

235170 rows × 152 columns

In [65]:
plus_f = pd.read_csv('../외감데이터/rate_final.csv')

In [66]:
plus_f['회계년도']

0         2019/12
1         2020/12
2         2021/12
3         2022/12
4         2019/12
           ...   
296285    2018/12
296286    2019/12
296287    2020/12
296288    2021/12
296289    2022/12
Name: 회계년도, Length: 296290, dtype: object

In [67]:
plus_f['회계년도'] = pd.to_datetime(plus_f['회계년도'], format='%Y/%m')

In [68]:
df = pd.merge(df, plus_f, on=['회사명','거래소코드','회계년도'], how='inner')


### 6. Labeling 진행

감사의견코드 : 'DI', 'DS', 'DU', 'AG', 'QS'
    부도: A
    정상: a

자본잠식률 : 100%
    부도: B
    정상: b

자본잠식률 : 50% (2년 연속)
    부도: C
    정상: c

In [69]:
# 회계년도를 datetime
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%Y-%m-%d')
df = df.sort_values(by=['거래소코드', '회계년도'])

# 감사의견코드 중 DI, DS DU, AG, QS가 존재하면 A 아니면 a
df['감사의견_y'] = df['감사의견코드'].apply(lambda x: "A" if x in ['DI', 'DS', 'DU', 'AG', 'QS'] else "a")


In [70]:
# 회계년도를 datetime
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%Y-%m-%d')
df = df.sort_values(by=['거래소코드', '회계년도'])

df['자본잠식률_100'] = df['자본잠식률'].apply(lambda x: "B" if x >= 100 else "b")

In [71]:
# 데이터셋을 거래소코드와 회계년도 기준으로 정렬
df.sort_values(by=['거래소코드', '회계년도'], inplace=True)

# 새로운 'Label' 컬럼을 기본값 'c'로 추가
df['자본잠식률_50'] = 'c'

# 이전 행의 값 초기화
prev_code = None
prev_cap_erode_rate = None

# 각 행을 순회하며 조건에 따라 'Label' 컬럼 설정
for index, row in df.iterrows():
    current_code = row['거래소코드']
    current_cap_erode_rate = row['자본잠식률']

    # 현재 행과 이전 행이 같은 회사이고, 두 해 모두 자본잠식률이 50 이상인 경우 'C'로 표시
    if current_code == prev_code and current_cap_erode_rate >= 50 and prev_cap_erode_rate >= 50:
        df.at[index, '자본잠식률_50'] = 'C'

    # 이전 행 값 업데이트
    prev_code = current_code
    prev_cap_erode_rate = current_cap_erode_rate

In [72]:
# 라벨링 조건에 따라 'y' 컬럼 생성
df['y'] = ((df['감사의견_y'] == 'A') | 
        (df['자본잠식률_100'] == 'B') | 
        (df['자본잠식률_50'] == 'C')).astype(int)

### 7. 부도년도 이후 데이터 삭제

197317 rows × 156 columns

In [73]:
def remove_post_default_data(group):
    # 최초 부도 년도 설정
    default_year = group[group['y'] == 1]['회계년도'].min()
    
    # default 값이 nan이 아니면 작거나 같은 데이터만 가져옴
    if not pd.isnull(default_year):
        group = group[group['회계년도'] <= default_year]
    
    return group

# 회사명으로 그룹바이 후 함수 적용
df = df.groupby('거래소코드').apply(remove_post_default_data).reset_index(drop=True)

### 8. 부도년도 이전 데이터

전년도의 자본잠식률이 50%면 1
나머지 데이터는 삭제

182114 rows × 156 columns

In [74]:
# 거래소 코드와 회계년도를 기준으로 정렬
df = df.sort_values(by=['거래소코드', '회계년도'])

for i in range(1, len(df)):
    # y가 1 인 경우
    if df.loc[i, 'y'] == 1:
        # 해당 행과 전 행의 거래소 코드가 같으면
        if df.loc[i, '거래소코드'] == df.loc[i-1, '거래소코드']:
            # 전 행의 자본잠식률이 50이상이면
            if df.loc[i-1, '자본잠식률'] >= 50:
                # 전 행의 y 가 1
                df.loc[i-1, 'y'] = 1

In [75]:
# 부도 데이터 중 정상 데이터 삭제

def remove_post_default_data(group):
    # 최초 부도 년도 설정
    default_year = group[group['y'] == 1]['회계년도'].min()
    
    # default 값이 nan이 아니면 크거나 같은 데이터만 가져옴
    if not pd.isnull(default_year):
        group = group[group['회계년도'] >= default_year]
    
    return group

# 회사명으로 그룹바이 후 함수 적용
df = df.groupby('거래소코드').apply(remove_post_default_data).reset_index(drop=True)

### 9. 중간 데이터 확인

# 전체 196204
# 정상 181631
# 부도 14573

196204 rows × 166 columns

In [76]:
df.to_csv('../외감데이터/dataset_부도11.csv')

In [77]:
df[df['y']==0]

,회사명,거래소코드,회계년도,감사의견코드,산업코드,산업명,설립일,기업규모코드,기업규모명,통계청 한국표준산업분류 코드 10차(대분류),...,new_유동자산증가율,new_재고자산증가율,new_매출액증가율,new_비유동자산증가율,new_순이익증가율,new_영업이익증가율,감사의견_y,자본잠식률_100,자본잠식률_50,y
0,(주)포스코건설,10000,2012-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c,0
1,(주)포스코건설,10000,2013-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-18.258059,-16.164419,14.016854,4.081387,-71.478710,1.363172,a,b,c,0
2,(주)포스코건설,10000,2014-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-12.449193,-8.092776,-6.396890,17.867219,-86.472926,-3.141584,a,b,c,0
3,(주)포스코건설,10000,2015-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,6.351912,-34.770304,-13.012219,-1.042386,127.038293,-1.159031,a,b,c,0
4,(주)포스코건설,10000,2016-12-01,UQ,64102,토목 건설업,1994/12/01,10.0,대기업,41,...,-5.085581,54.677241,-15.921085,-8.134513,-2616.490936,-17.380264,a,b,c,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196195,주식회사어라운드테이블,107400,2022-12-01,UQ,126802,부동산 관련 서비스업,2021/10/21,0.0,NaN,68,...,3247.443594,inf,inf,-88.591692,-4936.470588,-inf,a,b,c,0
196198,주식회사마커,107421,2021-12-01,NaN,126802,부동산 관련 서비스업,2013/01/11,NaN,NaN,68,...,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c,0
196199,주식회사마커,107421,2022-12-01,UQ,126802,부동산 관련 서비스업,2013/01/11,0.0,NaN,68,...,-26.218401,-100.000000,-33.930305,105.289701,-636.845293,-30.819887,a,b,c,0
196202,주식회사코차이나로지스틱스코리아,107471,2021-12-01,NaN,85209,기타 운송관련 서비스업,2009/01/19,NaN,NaN,52,...,NaN,NaN,NaN,NaN,NaN,NaN,a,b,c,0
